In [124]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from lgnpy import LinearGaussian
from sklearn.metrics import r2_score

![image](my_graph.png)

In [57]:
df_parameter=pd.read_csv("parameter.csv")
df_test=pd.read_csv("cyano_test.csv")
df_test=df_test.rename(columns={"PRECTOT": "prec", "ALLSKY_SFC_SW_DWN": "solar"})

In [58]:
df_test['DP']=df_test['DP'].apply(np.log)
df_test['TKN']=df_test['TKN'].apply(np.log)
df_test['DO']=df_test['DO'].apply(np.log)
df_test['Turb']=df_test['Turb'].apply(np.log)
df_test['Cyano']=df_test['Cyano'].apply(np.log)

In [60]:
df_parameter
mean = df_parameter['mean']
sd = df_parameter['sd']

In [65]:
lg = LinearGaussian()
lg.set_edges_from([('prec', 'dp'), ('prec', 'tkn'), ('dp', 'cyano'), ('tkn', 'cyano'),('solar','cyano'),('temp','cyano'),('do','cyano'),('turb','cyano')])
np.random.seed(42)
n=10000
data = pd.DataFrame(columns=['prec','dp','tkn','solar','temp','do','turb','cyano'])
data['prec'] = np.random.exponential(mean[0],n)
data['dp'] = mean[1]*data['prec']+np.random.normal(mean[2],mean[3],n)
data['tkn'] = mean[4]*data['prec'] + np.random.normal(mean[5],mean[6],n)
data['do'] = np.random.normal(mean[19],mean[20],n)
data['turb']=np.random.normal(mean[21],mean[22],n)
data['solar'] = mean[11]*np.random.normal(mean[7], mean[9],n)+mean[12]*np.random.normal(mean[8], mean[10],n)
data['temp'] = mean[17]*np.random.normal(mean[13], mean[15],n)+mean[18]*np.random.normal(mean[14], mean[16],n)
data['cyano']=mean[23]*data['solar']+mean[24]*data['temp']+mean[25]*data['tkn']+mean[26]*data['do']+mean[27]*data['dp']+mean[28]*data['turb']+np.random.normal(mean[29],mean[30],n)
lg.set_data(data)

In [66]:
df_test.head()

,Unnamed: 0,prec,solar,Date,DO,TKN,DP,Turb,Temp,Cyano
0,0,0.29,12.10,2008-08-18,1.678964,-0.504181,-2.407946,2.674149,24.6,9.427063
1,1,0.00,10.07,2009-12-16,2.448416,-0.581606,-2.407946,2.484907,0.9,0.000000
2,2,7.63,26.91,2010-07-14,1.837370,-0.476424,-2.120264,3.161247,27.1,5.030438
3,3,0.00,21.10,2012-09-19,2.073172,-0.414001,-2.120264,3.222868,20.4,5.509388
4,4,0.00,25.81,2007-08-15,1.919859,-0.360970,-2.813411,2.821379,27.9,7.876259


In [75]:
lg.set_evidences({'prec':0.29,'solar':12.10, 'do':1.678964,'tkn':-0.504181, 'dp':-2.407946, 'turb':2.674149, 'temp':24.6})
lg.run_inference(debug=False).loc['cyano','Mean_inferred']

6.6938

In [123]:
cyano_pred=[]
n=len(df_test)
for index, row in df_test.iterrows():
    lg.set_evidences({'prec':row['prec'],'solar':row['solar'], 'do':row['DO'],'tkn':row['TKN'], 'dp':row['DO'], 'turb':row['Turb'], 'temp':row['Temp']})
    cyano_pred.append(lg.run_inference(debug=False).loc['cyano','Mean_inferred'])
r2_score(df_test['Cyano'], cyano_pred)

/opt/anaconda3/lib/python3.8/site-packages/lgnpy/LinearGaussian.py:106: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  .replace(pd.np.nan, "", regex=True)


-0.731472293095274